# Tutorial 9:  Working with Geo-Data and PostGIS

## Introduction
In this tutorial, we will cover working with geo-spatial data, how to import it from different sources such as shape files or web APIs / GeoJSON, and how to store and query geo-spatial data in PostGIS.

We will also give a few hints and guidelines at the end of this notebook on which would be helpful tasks to solve for you as part of the practical assignment of a Cyclability study.

# EXERCISE 1 - Loading GeoSpatial Data from Shapefiles

We start with some small example of geo-spatial data avaialble in *shapefile* format. 

## YOUR TASK 1: Shape file download
We have prepared a small example data set about world countries. Your task is to
 - Download the corresponding shapefiles from Canvas (**wk9_data.zip**);
 - Uncompress the downloaded zip archive (should result in a data directory with 10 files); and 
 - Upload all included 10 files into your Jupyter server directory, **into a sub-directory *data* **
 

## Loading Shapefile data
Next, we want to load this shapfile with Python.

Geometry shape files can be parsed with Python using the **pyshp library**.

**Documentation**
pyshp library: https://github.com/GeospatialPython/pyshp

This library returns a **shapefile** object which then can be used to access either the actual **shapes** (geometries) of the file entries, or the associated data **records**.

The following Python code
   - opens the shapefile
   - prints how many shapes it contains
   - iterates through its ShapeRecords and prints each record, as well as which shape type it has and how many points the shape has.

In [ ]:
import shapefile
sf = shapefile.Reader("data/world.shp", encoding="iso-8859-1")

# which shpe type is it?
print(sf)

if (sf.shapeType == shapefile.POLYGON):
    shapes = sf.shapes()
    print("#shapes: ",len(shapes))
    for shape in shapes:
       print(shape.__geo_interface__['type'], len(shape.points), "points; bounding box:", shape.bbox)

The loaded shapefile data also included some additional attributes. 
Let's check which fields were loaded besides the actual shapes' geometry:

In [ ]:
# get the shape fields but skip first one which is a deletion flag used internally by dbf software
sf.fields[1:]

Above's output tells us that there are six fields for each shape record; for example, **pop_est** is the estimated population per country and has a numeric ('N') type; **name** is the country name with a minimum length of 0 and a maximum length of 80. 

Let's have now a look at the corresponding records in the shapefile, just the first ten:

In [ ]:
for i in range(0, 10):
    print(sf.record(i))

# EXERCISE 2: Writing Geo-Shaped Data to a Database and PostGIS Querying
Next, we want to see how we can store geometric data in PostgreSQL together with the other record data.

For the **PostgreSQL** database access, we will again use the **psycopg2** library which we introduced back in Week 4.
In addition, we also will work with PostgreSQL's PostGIS extensions.

### PostGIS
We installed PostGIS on one of our PostgreSQL servers.
**Important:** You need to use a different PostgreSQL server than we used so far:  

    soit-app-pro-1.ucc.usyd.edu.au

You can use the logins than you used so far:  

    username: 'y19s1d2x01_'+YOUR_UNIKEY  and pw your SID (student ID)

The old postgresql server does *not* support PostGIS.


Please edit in the following cell the variables **YOUR_UNIKEY** and **YOUR_PW** to match your own PostgreSQL login.

In [ ]:
import psycopg2
import psycopg2.extras

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = 'unikey'#'<your_unikey>'
    YOUR_PW     = '123456789'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                                database='y19s1d2x01_'+YOUR_UNIKEY,
                                user='y19s1d2x01_'+YOUR_UNIKEY,
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return conn

conn = pgconnect()

In the next step, we are looking into how to store the geo-data directly into a PostgreSQL database with PostGIS support. To do so, we copy the following utility function from Week 4 to execute an SQL command there from Python (actually, pgquery() is an improved version of pgexec() from Week 4).

In [ ]:
def pgquery( conn, sqlcmd, args=None, msg=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholders in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if msg != False:
                    print("success: " + msg)
            except psycopg2.DatabaseError as e:
                if e.pgcode != None and msg:
                    print("db read error: "+msg)
                    print(e)
            except Exception as e:
                print(e)
    return retval

Ok, now we are ready to proceed.

As usual with databases, we first have to create a schema for our shapefile data.
We derive this schema from the field names and types (as seen above) plus a geometry column for the actual shapes:

In [ ]:
world_schema = '''CREATE TABLE World (
                     pop_est NUMERIC, 
                     continent VARCHAR(80), 
                     name VARCHAR(80), 
                     iso_a3 VARCHAR(80), 
                     gdp_md_est NUMERIC,
                     geom GEOMETRY(Polygon,4326))''' 

pgquery(conn, "DROP TABLE World", msg="cleared old table")
pgquery(conn, world_schema, msg="created World table")

Notice the **geom** column at the end of the World table specification. This is where we will store the shapefile information for each country. The country's geometries are provided as polygons - we had checked this above in Exercise 1 - hence we give the *geom* attribute a type of **GEOMETRY(Polygon, 4326)**. The number **4326** is the *Spatial Reference Identifier* (SRID) of the WGS84 world geodetic coordinate system that is used by our example data set. This refers to GPS data. Be aware that not all geo-data will use the same reference system. In this case you must choose the correct SRID value - and if you compare geometries with different SRIDs, you need transform them first to the same SRID. This can be done (later) with the *ST_TRANSFORM()* function in PostGIS.

Once this table has been created, we want to load the world data into this table:

In [ ]:
import re

insert_stmt = """INSERT INTO World VALUES ( %(pop_est)s, %(continent)s, %(name)s, %(iso_a3)s, %(gdp_md_est)s,
                                            ST_GEOMFROMTEXT(%(geom)s, 4326) )"""

shapes = sf.shapes()
records= sf.records()

row = {}
for i in range(0, len(shapes)):
    record = sf.record(i)
    shape  = sf.shape(i)
    row['pop_est']=record[0]
    row['continent']=record[1]
    row['name']=record[2]
    row['iso_a3']=record[3]
    row['gdp_md_est']=record[4]
    print(row)
    
    # prepare the polygon data
    # this is a bit complex with our dataset as it has complex polygons, some with multiple parts...
    row['geom']="POLYGON(("
    i=0
    for x, y in shape.points:
       row['geom']+="%s %s," % (x,y)
       # check for start of a new polygon part
       i += 1
       if i in shape.parts:
           row['geom']= re.sub(",$", "),(", row['geom'])
    # properly end the polygon string
    row['geom'] = re.sub(",$", "))", row['geom'])
    
    # finally: insert new row into the table
    pgquery(conn, insert_stmt, args=row, msg="inserted "+str(record[2]))

Let's check what data we got loaded so far:

In [ ]:
query = "SELECT COUNT(*) FROM World"
result= pgquery(conn, query, returntype='dict')
from pprint import pprint 
print(result)

Next, we look at some example of spatial measurement functions in PostGIS:

The following example query finds *the five largest countries in Africa*.

What do you think - which African country is actually the largest?

In [ ]:
query = """SELECT name, ST_Area(geom) AS area 
             FROM World 
            WHERE continent='Africa' 
            ORDER BY area DESC 
            LIMIT 5"""
result= pgquery(conn, query, returntype='dict')
pprint(result)

Did you guess correctly? Ok, let's check which countries have a boundary with Algeria.

This is an example of a spatial join query where we compare country shapes that come from the same World table.
We can use one of the different spatial comparison functions which PostGIS supports, in particluar any of the topological relationships defined buy the *Nine Intersection Model* (cf. lecture slides). 

Normally, we would expect to use the *ST_TOUCH()* function here - but in our dataset, the boundaries are in low-resolution so that some border shapes actually intersect. Hence we go with the more generic *ST_INTERSECTS()* spatial relationship function:

In [ ]:
query = """SELECT B.name 
             FROM World A JOIN World B ON (ST_INTERSECTS(A.geom, B.geom))
            WHERE A.name = 'Algeria' AND B.name != A.name"""
result= pgquery(conn, query, returntype='dict')
pprint(result)

A complete list of the supported spatial measurement and relationship functions you find here:

https://postgis.net/docs/reference.html#Spatial_Relationships_Measurements

### Spatial Indexing
To make processing of such spatial joins a bit faster - especially if you are using larger data set - a spatial index is very helpful. PostgreSQL supports spatial indexing using its **GiST** index. This is documented here: https://postgis.net/docs/using_postgis_dbmanagement.html#idm2246

You can create an index on the country shape files with the following SQL command:

In [ ]:
index_command = "CREATE INDEX country_idx ON World USING GIST (geom)"
result= pgquery(conn, index_command, returntype='dict')
pprint(result)

Queries, especially complex queries and on larger data sets, using the World.geom attribute should now be a bit faster than without this index. try it out.

## YOUR TASK 2:  Experiment with the PostGIS topological functions

Following the above examples, answer the following questions using PostGIS:
 - Which countries have borders with Germany?
 - What are the five largest countries in Asia?
 - What are the three smallest in Asia?

In [ ]:
# TODO: Replace this cell with PostGIS SQL queries that 
#  - find the countries neighboring Germany, and 
#  - find the five largest countries in Asia, and
#  - find the three smallest countries in Asia
raise NotImplementedError

Never forget to clean up at the end:

In [ ]:
# When finished with Postgresql, do not forget to close the connection again:
conn.close()

## HOMEWORK Task: Spatial Join with Cities

The following is homework as there is not enough time to cover it in depth here in the tutorial. 
**Please continue with EXERCISE 3 during the tutorial lab time.**

We also have a cities data set made available on Canvas.
 - In the *wk9_data.zip* file, that you dopwnloaded earlier, there are also cities.shp files. Make sure that you have uploaded them too into your **data** subdirectory  on your Jupyter notebook server
 - Write the code to read the shapes from the Cities shapefile
 - Create a matching table in Postgresql  
 - Load the cities shapes into that table (tip: this should be easier than before because cities are represented as simple points rather than complex polygons).
 - Write a PostGIS query that determines which cities are located within the United States.

### HOMEWORK for Students in DATA2901 (Adv)
Solve the same questions from above using GeoPandas (Note: we do not have GeoPandas installed on the central Jupyter server; you hence will need to install GeoPandas on your own computer).
 - Which distinct continents are listed in the World data set?
 - Plot/map all European countries.
 - Plot/map the countries that are either in Europe and in Asia.
 - Can you write a new shapefile with just the European countries?

# EXERCISE 3 - Collecting Geo-Data for Neighbourhoods

In the practical assignment, you are asked to combine several datasets about *neighbourhoods*.
While the statistical data from the ABS share the same neighbourhood ID, the example dataset about transport options - the bike-sharing pods - does not have this ID, nor do the names match between bike-sharing pod locations and neighbourhoods.
The bike-sharing pods do have a GPS location though...

**Background:** This is a simulated dataset. Assume bicycle pods are dedicated places where publicly shared bicycles and scooters can be found. Users can take a bicycle or scooter from one station, cycle around, and return them at the same or at another location - similar to the public bike-sharing facilities in Melbourne, London, or New York. Users pay for the time duration of the bicycle usage.

In order to associate bicyle pods with the other data on neighbourhoods, we need to collect some spatial data for the neighbourhoods too. This is covered in the subsequent exercises in this tutorial.

Our goal in this first exercise is to extract the saptial boundaries of each Sydney neighbourhood (to be precise: of each SA2-level area in Greater Sydney) from the ABS website: http://stat.abs.gov.au/

The ABS collects census data in different granularities. We will work with the Statistical Area 2 (SA2) data which roughly corresponds to neighbourhoods of one or two suburbs. The level on top would be SA3 data. For example, *Darlinghurst* is a SA2 area which lies within the wider SA3 area *Sydney Inner City*, which in turn belongs to the SA4 area *Sydney - City and Inner South*, which is part of the Greater Capital City Statistical Area (GCCSA) of *Greater Sydney* in the state of *New South Wales*.

You can check the location and boundary of the various ABS areas here (choose "2016 Statistical Area Level 2 (SA2)" in the selector):
http://stat.abs.gov.au/itt/r.jsp?ABSMaps

In the following, we will look at two alternative approaches how to enrich the given ABS statistic data sets (from the assignment specification) with corresponding geo-boundaries per neighbourhood.

## Approach 1: Download SA2 Boundary Data
A first approach to get the actual boundary data is to download the boundary data shapefiles directly from here:

http://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.001July%202016?OpenDocument#Data

For example, on this page, the specific boundary data
**Statistical Area Level 2 (SA2) ASGS Ed 2016 Digital Boundaries MapInfo** is available directly in three different formats: **ESRI shapfile Format**, **TAB File Format**, and **MapInfo Interchange Format**.

These files can be downloaded and then parsed with Python using the **pyshp library**  (as discussed in the first exercise) - and loaded, for example, into PostgreSQL as we have done before. The PostGIS extension allows to store geo-data with the correct *spatial reference system* (Note: the SRID has to refer to **AGD94 in this case with data from the ABS**).

## YOUR TASK 3: Download and Inspect SA2 Boundary Shapefile

 - Download from ABS website the shapefile for australian SA2 areas (1270055001_sa2_2016_aust_shape.zip)
 - Upload the content of this zip archive to your folder on the jupyter server
 - Write a Python program to 
   - open the shapefile
   - print how many shapes it contains
   - iterate through its ShapeRecords and print each record, as well as which shape type it has and how many points the shape has.

In [ ]:
# TODO: download one of the boundary files from above's website
#       then iterate through the records in that file
#       and check its content -> and ultimately try to load into PostgreSQL
raise NotImplementedError

## Approach 2: Retrieve Boundary Data using Web APIs

In the second approach, we are looking at some examples on how to access web APIs which are specifically provided for program to retrieve data. 

These services use typically one of two different formats - either JSON or XML.

For **JSON**, we will use the standard language support in Python and its **request** library.<br>
For **XML**, we will use the **lxml** parser library.

### OpenStreetMap Web API
The OpenStreetMap Wep-API allows to convert a location address to a GPS location (and some information more, optionally even with full boundaries of suburbs or cities). The following example looks up the GPS location as well as the boundary polygon of the suburb 'Darlington' at "Darlington, New South Wales, Australia":

In [ ]:
# Lookup of a given address via OpenStreetMap Web-API:
import requests
import json
base_url = 'https://nominatim.openstreetmap.org/search'
my_params= {'q': 'Darlington, Sydney, NSW, Australia','format':'json','limit':'1'}
response = requests.get(base_url, params = my_params)

# let us see the raw JSON response
print(json.dumps(response.json(), indent=4, sort_keys=False))

Sometimes the submitted query address is not clearly specified and multiple potential locations might be returned. This is actually quite common, and OpenStreetMaps hence includes a **importance** value in its response which roughly represents how well a given location matches the given query string.

In the previous request, we actually only considered (and trusted) the first location returned - we had limited the response to just one match by setting a **limit:1** parameter. If you omit this limit, you can receive actually up-to 10 matches of various importance.

Let's look at this in the following example, where we made the query a bit more generic by just looking for a suburb or city 'Darlington', but without mentioning Sydney. At the same time, this also demonstrates how you can extract single values from a JSON web service response. This allows us to list the differnt matches in a more compact form than to pretty-dump the original longish JSON structure.

In [ ]:
# Lookup detailed boundary of a given address via OpenStreetMap Web-API with multiple potential matches
my_params= {'q': 'Darlington, NSW, Australia','format':'json'}
response = requests.get(base_url, params = my_params)
matches  = response.json()
num_matches = len(matches)

print(my_params['q'])
print("OpenStreetMaps returned", num_matches, "potential locations for the given query string.")
for i in range(0,num_matches):
    match_name     = matches[i]['display_name']
    match_latitude = matches[i]['lat']
    match_longitude= matches[i]['lon']
    #match_type     = matches[i]['type']
    #match_bounds   = matches[i]['boundingbox']
    match_confidence=matches[i]['importance']
    print(i, ".", match_name, match_latitude, match_longitude, " importance:", match_confidence)

The Web API from OpenStreetMap supports multiple different parameters:

| parameter | description |
| :--- | :--- |
| **format**=[**html** &vert; xml &vert; json &vert; jsonv2] | Output format, default = html|
| **q**=*query*                         | Query string to search for. Alternatively can be entered as:|
| &nbsp;                                | **street**=*housenumber* *streetname* |
| &nbsp;                                | **city**=*city* |
| &nbsp;                                | **county**=*county* |
| &nbsp;                                | **state**=*state* |
| &nbsp;                                | **country**=*country* |
| &nbsp;                                | **postalcode**=*postalcode* |
| **limit**=*integer*                   | Limit the number of returned results. Default is 10.|
| **polygon_geojson**=1                 | Output geometry of results in geojson format. |
| **polygon_kml**=1                     | Output geometry of results in kml format.|


Further deatils can be found here: https://wiki.openstreetmap.org/wiki/Nominatim

The following slightly extended example retrieves not just the bounding box, but the full detailed polygon boundary of Darlington from the OpenStreetMap API using an additional **polygon_geojson:1** parameter with the API request:

In [ ]:
# Lookup detailed boundary of a given address via OpenStreetMap Web-API with 'polygon_geojson' turned on
my_params= {'q': 'Darlington, Sydney, NSW, Australia','limit':'1','format':'json','polygon_geojson': '1'}
response = requests.get(base_url, params = my_params)

results      = response.json()
nhood_boundary_type = results[0]['geojson']['type']
nhood_boundary_exact= results[0]['geojson']['coordinates']
print(my_params['q'])
print("Location: ", nhood_geo_lat, nhood_geo_lon)
print("Exact Boundary: ", nhood_boundary_type, nhood_boundary_exact)

## YOUR TASK 4: Retrieve Geo-Locations and Boundaries for given Neighbourhoods

- Use the OpenStreetMap Web-Api to retrieve the *GPS boundary* of the neighbourhoods as listed in the assignment data in 'Neighbourhoods.csv' 
- For this you might need to inspect first how the JSON response is structured: Hence do first a pretty-print of the corresponding JSON response data.
- Tip: Please do not send too many requests too fast to the OpenStreetMap service. There is a request limit of 1 request/sec by this service which you need to honour to avoid to be blocked.
- Tip: if you want to see a map for a given GPS location, try: https://www.latlong.net/

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

# ASSIGNMENT PREPARATION 1: Writing Geo-Shaped Data to a Database
Lets finally store the extracted boundaries in PostgreSQL together with the other data about neighbourhoods.

For the **PostgreSQL** database output, use the **psycopg2** library as we used already in Week 4, as well as the new PostGIS server on  

**soit-db-pro-1.ucc.usyd.edu.au**

## YOUR TASK: Store Neighbourhood Boundaries in Database
Your task is as follows:
* Extend the table with your neighbourhood data, which matches the CSV files from Canvas about assignment 1, so that it can also store the geo-boundaries of a neighbourhood.
* Insert/Update the extracted information of the neighbourhood boundaries in that table
* Be careful to specify the correct SRID which matches your dataset (depending on whether it is coming from the ABS or OpenStreetMap; both use different spatial reference systems).
* Tip: Whenever possible, use PostGIS (soit-db-pro-1.ucc.usyd.edu.au) which is documented here:  https://postgis.net/docs/
* Postgresql has also some internal geometry data types which can be helpful too - though be careful, they do not support different spatial reference systems, hence are not really suitable for geo-data, especially if this data comes from different sources with different coordinate systems: https://www.postgresql.org/docs/9.5/static/datatype-geometric.html

**Note:** as this is essentially an sub-task of the practical assignment, we will not provide an example solution for this step.

In [ ]:
# TODO: replace the content of this cell with your Python + SQL solution
raise NotImplementedError

# ASSIGNMENT PREPARATION 2: Spatial Join between Bike-sharing Pods and Neighbourhoods

The next step in your assignment would then be a spatial join between the bike-sharing locations and the neighbourhood boundaries.

## YOUR TASK: Spatial Join between bike-sharing pods and neighbourhoods
Your task is as follows:
* Check with the postgresql documentation on how to do a spatial join
* Query in SQL for each bike-sharing pod in which neighbourhood it is located; if you have the boundaries then you can check for containment; otherwise you might need to look for the shortest distance between the pod location and the centre of a neighbourhood.
* It might be a good idea to store the computed relationships between pods and neighbourhoods in the database too.
* Tip: PostgreSQL supports a number of spatial functions for points and other geometry types: https://www.postgresql.org/docs/9.5/static/functions-geometry.html 
* Tip 2: We also have installed the PostGIS extension on the  soit-db-pro-1.ucc.usyd.edu.au  server.
   This allows for more control and native support for geo-spatial data. Documentation of PostGIS: http://postgis.net/documentation
* Be aware of the different spatial reference systems used by the ABS data and by our own provided data or OpenStreetMap. You might need to use a transformation first before comapring geo-data from different sources using the *ST_TRANSFORM()* function of PostGIS (https://postgis.net/docs/ST_Transform.html).
* If you have a lot of data, this spatial join definitly will benefit from a spatial index...
   
   
**Note:** as this is essentially an sub-task of the practical assignment, we will not provide an example solution for this step.

In [ ]:
# TODO: replace the content of this cell with your Python + SQL solution
raise NotImplementedError

# References

Books:
- Seppe van den Broucke and Bart Baesens: "Practical Web Scraping for Data Science", Springer 2018. (available electroinically via USYD library)
- PostGIS: http://postgis.net/documentation
- Shapefile import: https://github.com/GeospatialPython/pyshp
- OpenStreetMap Web API:  https://wiki.openstreetmap.org/wiki/Nominatim

# End of Tutorial. Many Thanks.